In [0]:
-- Find customers who placed orders above the average order amount.


select distinct c.first_name, c.last_name , o.total_amount
FROM customers c INNER JOIN orders o ON c.customer_id = o.customer_id WHERE o.total_amount >
(
SELECT AVG(total_amount) FROM orders
)
     

In [0]:
-- List all products that have been sold more than 5 times in total.
SELECT
od.product_name as PRODUCT_NAME,sum(od.quantity) AS TOTAL_SOLD
FROM order_details od GROUP BY od.product_name HAVING SUM(od.quantity) >
(SELECT 2)

In [0]:
--  List customers and their total quantity of products ordered.

WITH customer_orders AS (
    SELECT c.customer_id, c.first_name, SUM(od.quantity) AS total_quantity
    FROM customers c
    INNER JOIN orders o ON c.customer_id = o.customer_id
    INNER JOIN order_details od ON o.order_id = od.order_id
    GROUP BY c.customer_id, c.first_name
)

SELECT * FROM customer_orders WHERE total_quantity>3;

In [0]:
%sql
--Find products with their total sales revenue (price * quantity).



WITH product_sales AS (
    SELECT od.product_name, SUM(od.price_each * od.quantity) AS total_revenue
    FROM order_details od 
    GROUP BY 1
)

SELECT * FROM product_sales WHERE total_revenue > 10000

CASE Statements (SQL IF/ELSE)


In [0]:
--Categorize each order as 'Small', 'Medium', 'Large'.


SELECT o.order_id,o.total_amount, 
      CASE
          WHEN o.total_amount<1000 THEN 'SMALL'
          WHEN o.total_amount BETWEEN 1000 and 5000 THEN 'MEDIUM'
          ELSE 'LARGE'
      END AS ORDER_SIZE 


FROM orders o 

In [0]:
-- Categorize each order as 'Small', 'Medium', 'Large'.

WITH categorized_orders AS (
    SELECT o.order_id, o.total_amount, 
          CASE
              WHEN o.total_amount < 1000 THEN 'SMALL'
              WHEN o.total_amount BETWEEN 1000 AND 1500 THEN 'MEDIUM'
              ELSE 'LARGE'
          END AS ORDER_SIZE 
    FROM orders o
)
SELECT * 
FROM categorized_orders 
WHERE ORDER_SIZE = 'LARGE'

In [0]:
--Label customers as 'New' or 'Returning' based on order count.


SELECT c.first_name, COUNT(o.order_id) AS total_orders,
       CASE
           WHEN COUNT(o.order_id) = 1 THEN 'New'
           WHEN COUNT(o.order_id) > 1 THEN 'Returning'
           ELSE 'No Orders'
       END AS customer_type
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name;

FUNCTIONS (STRIING + DATE)

In [0]:
SELECT UPPER(c.first_name) AS FIRST_NAME_UPPERCASE from customers c 


In [0]:
SELECT LOWER(c.first_name) AS FIRST_NAME_UPPERCASE from customers c

In [0]:
SELECT UPPER(CONCAT(c.first_name," ",c.last_name)) AS FULL_NAME from customers c 

In [0]:
-- Extract month of each order & group orders by month.
SELECT MONTH(o.order_date) AS order_month, count(o.order_id) AS _total_orders 
FROM orders o 
GROUP BY 1
ORDER BY 1

In [0]:
-- List each customer's name + their total spend + spending category.

WITH customer_spend AS (
    SELECT c.customer_id, CONCAT(c.first_name, ' ', c.last_name) AS full_name,
           SUM(od.quantity * od.price_each) AS total_spent
    FROM customers c
    INNER JOIN orders o ON c.customer_id = o.customer_id
    INNER JOIN order_details od ON o.order_id = od.order_id
    GROUP BY c.customer_id, c.first_name, c.last_name
)

SELECT full_name, total_spent,
       CASE
           WHEN total_spent < 10000 THEN 'Bronze'
           WHEN total_spent BETWEEN 10000 AND 2000 THEN 'Silver'
           ELSE 'Gold'
       END AS customer_tier
FROM customer_spend;
     